# Gurobi primer v Python

Primeri so povzeti po hitrem vodiču za Gurobi, dostopni na [Gurobi Quick Start Guides](https://www.gurobi.com/documentation/quickstart.html).

V kolikor je bila namestitev __gurobipy__ uspešna, bi spodnji ukaz moral spisati kratko pomoč o vmesniku.

In [1]:
import numpy as np
import gurobipy as gp
import scipy.sparse as sp
from gurobipy import GRB

gp.help()


  Gurobi shell.  Available top-level commands are:
    read(): Read a model from a file (e.g., m = read("junk.mps")
            typically followed by m.optimize())
    models(): Provide a list of the currently loaded models.
    setParam(): Change a parameter setting (e.g., setParam("NodeLimit", 10)).
    readParams(): Read parameter settings from a file.
    resetParams(): Reset parameters to their default values.
    paramHelp(): Get help on a parameter (e.g., paramHelp("NodeLimit")).
    writeParams(): Write current parameter settings to a file.
    system(): Perform a shell command (e.g., system("ls")).

  Additional help on each of these commands may be obtained by typing
  "help(command)" (e.g., help(setParam)).

  Additional commands are also available for the basic objects that are
  created and manipulated within the Gurobi optimizer.  More help on these
  commands can be obtained by typing:
    help(Model)     : the Gurobi model object
    help(Var)       : the Gurobi variab

## Enostaven primer:
maksimiziraj 
```
x + y + 2z
```
z omejitvami
```
x + 2y + 3z <= 4
x + y >= 1
x, y, z so binarne
```

In [2]:
try:
    # Ustvari model 
    m = gp.Model("enostavenModel")
    # Ustvari spremenljivke
    x = m.addVar(vtype=GRB.BINARY , name="x")
    y = m.addVar(vtype=GRB.BINARY , name="y")
    z = m.addVar(vtype=GRB.BINARY , name="z")
    # Nastavi cenitveno funkcijo
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)
    # Podaj omejitve
    m.addConstr(x + 2 * y + 3 * z  <= 4, "c0")
    m.addConstr(x + y  >= 1, "c1")
    # Pozeni optimizacijo modela
    m.optimize()
    
    print('\n-------------------')
    print('Najdena resitev:')
    for v in m.getVars():
        print(f'{v.varName}={v.x}')
    print(f'vrednost: {m.objVal}')
except gp.GurobiError  as e:
    print(f'Napaka {e.errno}: {e}')
except AttributeError:
    print('Napaka v atributih')

Academic license - for non-commercial use only - expires 2021-02-24
Using license file /Users/david/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xf43f5bdf
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%

-------------------
Najdena resitev:
x=1.0
y=0.0
z=1.0
vrednost: 3.0

## Uporaba z matrikami
Podobno kot v MATLAB (ali R) lahko Gurobi uporabimo tudi s matrikami. Rešimo isti primer kot zgoraj z uporabo matrik:

In [3]:
try:
    # Ustvari model 
    m = gp.Model("matricniModel")
    # Ustvari spremenljivke
    x = m.addMVar(shape=3, vtype=GRB.BINARY, name="x")
    
    # Nastavi cenitveno funkcijo
    f = np.array([1.0, 1.0, 2.0])
    m.setObjective(f @ x, GRB.MAXIMIZE)
    
    # Redka matrika za levo stran ... preuredili smo drugo omejitev iz >= v <=
    val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
    # V kateri vrstici in stolpcu se nahaja val[i]? -> row[i] in col[i]
    row = np.array([0, 0, 0, 1, 1]) # vrstice za vrednosti
    col = np.array([0, 1, 2, 0, 1]) # stolpci za vrednosti
    # Ustvari matriko
    A = sp.csr_matrix((val, (row, col)), shape=(2, 3))
    # Vektor na desni strani
    b = np.array([4.0, -1.0])
    # Uvedi omejitve v model
    m.addConstr(A @ x <= b, name="c")
    
    # Pozeni optimizacijo modela
    m.optimize()
    
    print('\n-------------------')
    print('Najdena resitev:')
    print(f'x={x.X}')
    print(f'vrednost: {m.objVal}')
except gp.GurobiError  as e:
    print(f'Napaka {e.errno}: {e}')
except AttributeError:
    print('Napaka v atributih')

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x6d8be88c
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%

-------------------
Najdena resitev:
x=[1. 0. 1.]
vrednost: 3.0


## Uporaba LP formata
__gurobipy__ podpira tudi LP (in MPS) formate. Primer reševanja problemov s kovanci (glej poročilo in poglavje _2.2.1 - Opis problema_ za podrobnosti):

In [4]:
try:
    coins_path = "coins.lp"
    # Preberi model iz datoteke 
    m = gp.read(coins_path)
    
    # Pozeni optimizacijo modela
    m.optimize()
    
    print('\n-------------------')
    print('Najdena resitev:')
    for v in m.getVars():
        print(f'{v.varName}={v.x}')
    print(f'vrednost: {m.objVal}')
except gp.GurobiError  as e:
    print(f'Napaka {e.errno}: {e}')
    print(f'Ali je pot do datoteke "{coins_path}" pravilna?')
except AttributeError:
    print('Napaka v atributih')

Read LP format model from file coins.lp
Reading time = 0.00 seconds
: 4 rows, 10 columns, 18 nonzeros
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 10 columns and 18 nonzeros
Model fingerprint: 0x90cc2da6
Variable types: 4 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+01]
  Objective range  [1e-02, 1e+00]
  Bounds range     [5e+01, 1e+03]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 9 nonzeros
Variable types: 0 continuous, 4 integer (0 binary)

Root relaxation: objective 1.149294e+02, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  114.92942    0    1   -0.00000  114.